# Otimização Natural - 4ª Lista de exercícios

Aluno: Fernando Dias

A primeira parte dessa lista de exercícios possui um código auxiliar escrito por mim e está disponível no repositório https://github.com/Fdms-3741/OtimizacaoNaturalLista) como lista4. 

A segunda parte explora o uso da biblioteca DEAP (https://deap.readthedocs.io/https://deap.readthedocs.io/) para a resolução de problemas de evolução.

## Estrutura do meu código para essa lista

Nessa seção, será feito uma breve descrição da estrutura geral do código e como ela é utilizada ao longo da 

A base do projeto está no arquivo e `ea.py` que implementa a classe base `EvolutionaryAlgorithm`. Essa classe define métodos abstratos que representam os passos gerais a serem tomados por qualquer algoritmo genético. São eles:

* Seleção dos Pais
* Recombinação
* Mutação
* Seleção dos sobreviventes

Essa classe também implementa a função de execução do algoritmo evolucionário e funções auxiliares; como por exemplo para a cronometragem do algoritmo, controle de parâmetros do algoritmo e exportação dos resultados em um relatório.

Classes como `GenecticAlgorithm`, `EvolutionaryStragegy` e `EvolutionaryProgramming` implementam algoritmos genéticos, estratégias evolutivas e programação evolucionária a partir da definição base de `EvolutionaryAlgorithm`. Essas classes implementam a criação de indivíduos e implementam as funções que foram definidas acima. 

Cada questão é programada a partir de uma das três classes apresentadas anteriormente e deve apenas implementar a função de cálculo de aptidão que é exclusiva de cada problema. Caso seja necessário, a função também pode reimplementar quaisquer funções anteriormente mencionadas.

Em cada questão, serão definidos os comportamentos das funções a medida que aparecerem ou forem modificadas.

## Questão 1 - Simple Genectic Algorithm (SGA) 

O objetivo dessa questão é otimizar a função $f(x) = x^2-0.3\cos(10\pi x)$. 

### Construção do SGA 

Para isso, define-se a classe `GenecticAlgorithm` que tem como operações o seguinte:

* Seleção de pais: São selecionados metade dos indivíduos com base numa roleta onde a aptidão é a probabilidade de sorteio.
* Recombinação: São sorteados $2\mu$ pares aleatoriamente para serem recombinados. A recombinação consiste no sorteio aleatório de um trecho de bits e alternado entre os pais. Os trechos são alternados em função da probabilidade de crossover $p_c$.
    * Exemplo: um pai é representado por $\vec{x}=\begin{bmatrix}x_0&x_1&x_2&x_3&x_4&x_5\end{bmatrix}$ e outro pai é representado por $\vec{y}=\begin{bmatrix}y_0&y_1&y_2&y_3&y_4&y_5\end{bmatrix}$. É sorteado que os elementos 2, 3 e 4 serão trocados entre pais. Então os filhos serão $\vec{x}'=\begin{bmatrix}x_0&x_1&y_2&y_3&y_4&x_5\end{bmatrix}$ e $\vec{y}'=\begin{bmatrix}y_0&y_1&x_2&x_3&x_4&y_5\end{bmatrix}$.
* Mutação: São sorteados os filhos que vão sofrer a mutação em função de $p_m$. Para cada filho sorteado, cada bit do filho pode ser trocado ou não em função de $p_m$.
* Seleção dos sobreviventes: Sobrevivem apenas os $\mu$ indivíduos com maior aptidão.

Abaixo, será demonstrado como realizar uma execução do algoritmo genético utilizando a questão 1 como exemplo. A nova classe será a `Questao1` e ela conterá a função que calcula a aptidão e retorna o resultado.

In [1]:
import numpy as np 
from sga import GeneticAlgorithm

def ConvertBitsArrayToInteger(array):
    Y,_ = np.meshgrid(np.arange(array.shape[1]),np.arange(array.shape[0]))
    Y = 2**Y
    result = np.where(array,Y,0)
    return np.sum(result,axis=1)

class Questao1(GeneticAlgorithm):
    
    def CalculateFenotypes(self,individuals):
        """
        Calcula o valor de cada indivíduo
        """
        values = ConvertBitsArrayToInteger(individuals)
        values = values-2**(self.bitsSize - 1)
        values = values/2**(self.bitsSize - 2)
        return values

    def CalculateAptitude(self,individuals):
        """
        Calcula a aptidão de cada indivíduo
        """
        values = self.CalculateFenotypes(individuals)
        return - (np.power(values,2)-0.3*np.cos(10*np.pi*values))
     

Faremos 5 execuções do algoritmo para um número diferente de passos. A tabela será gerada pelo código abaixo:

In [2]:
import pandas as pd

execucoes = []
resultados = []
for execucao in [1,5,10,20,40]:
    for i in range(5):
        # Parâmetros de execução
        a = Questao1({
            "populationSize": 30, # Tamanho da população
            "bitsSize": 18, # Tamanho dos bits
            "mutationProbability": 0.7, # Probabilidade de mutação
            "crossoverProbability": 0.3, # Probabilidade de crossover
            "numberOfSelectedParents":15, # Número de pais selecionados
            "numberOfRecombinedOffspring":60 # Número de filhos gerados
        })
        a.progress = False
        a.Execute(10) # Executa N passos
        report = a.Report() # Expõe resultado
        execucoes.append(execucao)
        resultados.append(report['Results']['Best aptitude'])
        
resultados = pd.DataFrame({
    'Passo':execucoes,
    'Melhor aptidão':resultados
})

resultados.groupby('Passo')['Melhor aptidão'].describe()[['mean','std']]

,mean,std
Passo,,
1,0.298881,0.001478
5,0.291512,0.015399
10,0.295520,0.005312
20,0.297105,0.003823
40,0.298577,0.002077


Vemos que apenas para cinco execuções o algoritmo já tem um desempenho ótimo.

## Questão 4 - Mais problemas para SGA

Nessa questão, o mesmo algoritmo de SGA é utilizado em diferentes problemas. Cada uma deles será resolvido com a criação de uma nova classe que sobrescreve a função de aptidão para calcular o resultado do problema.

Para o problema das $N$ rainhas e o problema do caixeiro viajante, foi criada a classe `ProblemaPermutacaoInteiros` que modifica as operações de geração de indivíduos, recombinação e mutação para suportar problemas de espaço de permutação. Essa classe implementa o _cycle crossover_ para a recombinação e faz a troca de um único elemento para a mutação. 

Vemos a classe implementada abaixo:

In [3]:
class ProblemaPermutacaoInteiros(GeneticAlgorithm):
    
    def GenerateIndividuals(self):
        # Cria um vetor de N indivíduos que vai de 0 até N-1
        Y,_ = np.meshgrid(np.arange(self.bitsSize),np.arange(self.populationSize))
        for i in range(Y.shape[0]):
            np.random.shuffle(Y[i])
        self.individuals = Y

    def RecombineParents(self,parent1,parent2):
        """
        Implementa um cycle crossover simples: Ele apenas alterna os elementos de um único ciclo
        """
        offspring = np.zeros((2,self.bitsSize))
        cycle = np.zeros(self.bitsSize)
        startIndex = 0
        currentIndex = startIndex 
        while True:
            cycle[currentIndex] = 1
            # Qual o índice do valor no pai 1 no qual 
            #o valor é idêntico ao valor presente no indice atual do pai 2
            currentIndex = np.argwhere(parent1 == parent2[currentIndex])[0][0]
            # Se o índice voltou ao valor original, quebra-se o loop
            if currentIndex == startIndex:
                break
        offspring[0],offspring[1] = np.where(cycle,parent1,parent2), np.where(cycle,parent2,parent1)
        return offspring
            
    def Mutation(self,individuals):
        """
        A mutação apenas troca a posição entre dois elementos do indivíduo, se selecionado
        """
        pm = self.mutationProbability
        mutationChoice = np.random.choice([0,1],p=[1-pm,pm],size=(individuals.shape[0]))
        for idx in np.argwhere(mutationChoice):
            idx = idx[0]
            #__import__('pdb').set_trace()
            i,j = np.random.choice(individuals.shape[1],replace=False,size=(2,))
            individuals[idx][i], individuals[idx][j] = individuals[idx][j],individuals[idx][i]
        return individuals

### Problema das N rainhas

O problema das N rainhas consiste em um tabuleiro de tamanho $N\times N$ na qual se quer encontrar um posicionamento para $N$ rainhas de modo que nenhuma peça possa capturar nenhuma outra peça. 

Fazemos a representação desse problema na qual o tabuleiro é representado por um vetor $N$ onde cada elemento representa uma coluna. Cada elemento $i$ possui um número $j$ que representa a posição da $i$-ésima rainha está na célula $i\times j$. Para não haver conflitos entre linhas, os elementos devem ser únicos. Assim, temos um vetor de tamanho $N$ com os números de $0$ a $N-1$ distribuídos entre os elementos, e o objetivo é permutar esses números de modo que não haja capturas possíveis nas diagonais.

Para determinar se uma captura ocorre na diagonal. Deve a partir do elemento atual $x_i$ validar se os elementos a $k$ colunas de distância da colua atual são $x_i+k$ ou $x_i-k$. Soma-se toda a vez que uma captura é detectada. O ótimo portanto deve ser 0.

Para esse problema, implementa-se uma nova função aptidão com a classe `ProblemaRainhas`:

In [4]:
class ProblemaRainhas(ProblemaPermutacaoInteiros):
    
    def CalculateAptitude(self,individuals):
        # Calcula aptidão com piores resultados possíveis
        aptitudeList = np.zeros(individuals.shape[0])

        # Para cada indivíduo, um custo
        for idx, individual in enumerate(individuals):
            cost = 0
            # Para cada elemento
            for posIdx, element in enumerate(individual):
                #
                for i in range(individual.shape[0]):
                    if i == posIdx:
                        continue
                    if element == individual[i] - (i-posIdx) or element == individual[i] + (i-posIdx):
                        cost -= 1

            aptitudeList[idx] =  cost

        return aptitudeList

a = ProblemaRainhas({
            "populationSize": 100, # Tamanho da população
            "bitsSize": 8, # Tamanho dos bits
            "mutationProbability": 0.7, # Probabilidade de mutação
            "crossoverProbability": 0.7, # Probabilidade de crossover
            "numberOfSelectedParents":50, # Número de pais selecionados
            "numberOfRecombinedOffspring":300 # Número de filhos gerados
})
a.progress = False
a.Execute(100) # Executa N passos
print("Valor ótimo:",0)
print("Valor encontrado:",a.Report()['Results']['Best aptitude'])

Valor ótimo: 0
Valor encontrado: 0.0


### Problema do caixeiro viajante

Para o problema do caixeiro viajante, tem-se $P$ cidades na qual deve-se planejar uma rota que passe por todas elas com o menor custo possível. 

A representação desse problema requer também a representação do espaço permutação para resolver o problema. Nesse caso cada resposta corresponde com a ordem na qual as cidades são visitadas. O custo é calculado ao se tirar ao somar o módulo da diferença entre o vetor posição da cidade $i$ com a cidade $i+1$ para todo $i\in\{0,\dots,P-1\}$.

Para testar a implementação, cria-se uma distribuição de cidades equidistantes no entorno do círculo unitário. O valor ótimo é o valor mais próximo do comprimento da circunferência.

In [98]:
class CaixeiroViajante(ProblemaPermutacaoInteiros):
    
    def CalculateAptitude(self,individuals):
        # Calcula aptidão com piores resultados possíveis
        aptitudeList = np.zeros(individuals.shape[0])
        #print(individuals)
        # Para cada indivíduo, um custo
        for idx, individual in enumerate(individuals):
            aptitudeList[idx] = - np.sum(np.linalg.norm(self.posicoes[individual[1:]]-self.posicoes[individual[:-1]],axis=1))
            
        return aptitudeList
        
    def Report(self):
        """
        Gera um relatório para cada execução do algoritmo
        """
        report = {}
        report['Creation time'] = float(self.creationTime)
        report['Current execution'] = self.executeDuration
        report['SGA Params'] = {}
        for param in self.sgaParams:
            report['SGA Params'][param] = float(getattr(self,param))
        report['Results'] = {}
        report["Results"]['Best Individual'] = [int(i) for i in self.bestFitIndividual]
        #report['Results']['Best Individual value'] = float(self.CalculateFenotypes(self.bestFitIndividual[np.newaxis,:])[0])
        report['Results']["Best aptitude"] = float(self.bestFitAptitude)
        return report 

a = CaixeiroViajante({
    "populationSize": 100, # Tamanho da população
    "bitsSize": 30, # Tamanho dos bits
    "mutationProbability": 0.7, # Probabilidade de mutação
    "crossoverProbability": 0.7, # Probabilidade de crossover
    "numberOfSelectedParents":50, # Número de pais selecionados
    "numberOfRecombinedOffspring":300 # Número de filhos gerados
})
a.progress = False
a.posicoes = np.stack([np.cos(2*np.pi*np.arange(30)/30),np.sin(2*np.pi*np.arange(30)/30)]).T
a.Execute(1000)
optimalValue = a.CalculateAptitude(np.arange(30)[np.newaxis,:])
print("Valor ótimo:",optimalValue)
print("Valor encontrado:",a.Report()['Results']['Best aptitude'])

Valor ótimo: [-6.06265087]
Valor encontrado: -7.191855155706312


### Problema de clusterização 

O problema de clusterização consiste em determinar qual o melhor particionamento de dados que minimiza a distância dos dados até os centróides. Temos que os $N$ dados tem $M$ dimensões e devem ser particionados entre $P$ clusters.

Para esse problema, o espaço de decisão é numérico e consiste em um vetor $x$ de tamanho $N$ onde cada elemento indica a qual cluster um dado 

In [6]:
M = 2
N = 100
P = 10

class ProblemaCluster(ProblemaPermutacaoInteiros):

    def GenerateIndividuals(self):
        self.clusters = P
        # Cria um vetor de N indivíduos que vai de 0 até N-1
        self.individuals = np.random.choice(self.clusters,size=(self.populationSize,self.bitsSize))
            
    def Mutation(self,individuals):
        """
        A mutação apenas troca a posição entre dois elementos do indivíduo, se selecionado
        """
        pm = self.mutationProbability
        mutationChoice = np.random.choice([0,1],p=[1-pm,pm],size=(individuals.shape[0]))
        for individualIdx in np.argwhere(mutationChoice):
            individualIdx = individualIdx[0]
            mutateValues = np.random.choice([0,1],p=[1-pm,pm],size=(individuals.shape[1]))
            for mutationIdx in np.argwhere(mutateValues):
                mutationIdx = mutationIdx[0]
                individuals[individualIdx][mutationIdx] = np.random.randint(self.clusters)
            
        return individuals



## Questão 2 - Função de Ackley

Para a função a seguir, decidiu-se utilizar a biblioteca [DEAP](https://deap.readthedocs.io/en/master/) para criação de algoritmos de estratégia evolutiva para resolução da função de Ackley.

A intenção de uso da biblioteca é explorar as funcionalidades para facilitar implementações futuras e auxiliar na construção de código para o trabalho.

Abaixo, temos o código que fará a o seguinte:
* Define o indivíduo e a estratégia
* Define a função que gera o indivíduo
* Define estratégias de recombinação e mutação como vistas em ES
* Implementa a função de ackley como função aptidão
* Executa o algoritmo evolucionário para 100 de população

In [103]:
from deap import base, creator, tools, algorithms
import random
import array


# Probabilidade de mutação
pm = 0.7
individualSize = 2


toolbox = base.Toolbox()

# Define que o problema é de minimização (objetivo invertido com peso -1)
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# Cria o indivíduo e separa o genótipo entre termos e estratégia, ambos são do tipo decimal
creator.create("Individual", array.array, typecode="d", fitness=creator.FitnessMin, strategy=None)
creator.create("Strategy", array.array, typecode="d")

# Gera valores tanto para o indivíduo quanto para a estratégia
def generateES(icls, scls, size, imin, imax, smin, smax):
    ind = icls(random.uniform(imin, imax) for _ in range(size))
    ind.strategy = scls(random.uniform(smin, smax) for _ in range(size))
    return ind

# Registra o indivíduo, tamanho e as fronteiras (min, max) dos valores da função e estratégias
toolbox.register("individual", generateES, creator.Individual, creator.Strategy,
    individualSize,-31, 31, 1, 100)

# Define um decorador que força um valor mínimo para a parcela de estratégia
def checkStrategy(minstrategy):
    def decorator(func):
        def wrappper(*args, **kargs):
            children = func(*args, **kargs)
            for child in children:
                for i, s in enumerate(child.strategy):
                    if s < minstrategy:
                        child.strategy[i] = minstrategy
            return children
        return wrappper
    return decorator

toolbox.register("population", tools.initRepeat, list, toolbox.individual)
# Estratégia de recombinação
toolbox.register("mate", tools.cxESTwoPoint)
# Estratégia de mutação
toolbox.register("mutate", tools.mutESLogNormal, c=5.,indpb=pm)
# Adiciona nas duas etapas a condição de valor mínimo para a estratégia
toolbox.decorate("mate", checkStrategy(0.1))
toolbox.decorate("mutate", checkStrategy(0.1))

# Estratégia de seleção de sobreviventes: Roleta
toolbox.register("select",tools.selRoulette)
def Ackley(individual):
    a = 20
    b = 0.2
    c = 2*np.pi
    ind = np.array(individual)
    d = ind.shape[0]
    assert d == individualSize
    return - a * np.exp(-b*np.sqrt(np.sum(np.power(ind,2))/d)) - np.exp(np.sum(np.cos(c*ind))/d) + a + np.exp(1),
                    
toolbox.register('evaluate',Ackley)
# Estratégia

def main():
    MU, LAMBDA = 30, 600
    pop = toolbox.population(n=MU)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("std", np.std)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, logbook = algorithms.eaMuCommaLambda(pop, toolbox, mu=MU, lambda_=LAMBDA, 
        cxpb=0.2, mutpb=0.8, ngen=1000, stats=stats, halloffame=hof,verbose=False)

    return pop, logbook, hof
pop,logbook,hof = main()


No fim da execução, temos os seguintes resultados:

In [104]:
# Melhor indivíduo
hof[0]

Individual('d', [-1.9773074417134806, -1.2568544249106055])

In [105]:
# Valor da função ackley encontrada
hof[0].fitness.values

(6.753670595842246,)

In [106]:
# Valor da estratégia
hof[0].strategy

Strategy('d', [3.5280882076697697, 8.62135919904436])

Vemos que o algoritmo não foi capaz de encontrar o valor ótimo, mas se aproximou consideravelmente.